In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tf_keras.preprocessing.image import ImageDataGenerator
import tf_keras

2024-08-28 22:35:32.410865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 22:35:32.424511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 22:35:32.436252: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 22:35:32.439216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 22:35:32.454174: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (299, 299))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

full_images, full_labels = load_images_from_folder('Full', 0)
half_images, half_labels = load_images_from_folder('Half', 1)
low_images, low_labels = load_images_from_folder('Low', 2)
no_images, no_labels= load_images_from_folder("Absent", 3)

images = np.concatenate((full_images, half_images, low_images, no_images))
labels = np.concatenate((full_labels, half_labels, low_labels, no_labels))
images = images / 255.0

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [4]:
resnet_v2= "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/101-classification/2"

model= tf_keras.Sequential([
    hub.KerasLayer(
        resnet_v2, 
        input_shape= (299, 299, 3), 
        trainable= False
        ),
        tf_keras.layers.Dense(128, activation='relu'),
        tf_keras.layers.Dropout(0.5),
        tf_keras.layers.Dense(4, activation='softmax')
])

with tf.device("/GPU:0"):
    model.compile(
        optimizer= "adam", 
        loss= "sparse_categorical_crossentropy",
        metrics= ["accuracy"]
    )

model.summary()

I0000 00:00:1724864856.353933    5480 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864856.376793    5480 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864856.376845    5480 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864856.379170    5480 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724864856.379214    5480 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              44677609  
                                                                 
 dense (Dense)               (None, 128)               128256    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 44806381 (170.92 MB)
Trainable params: 128772 (503.02 KB)
Non-trainable params: 44677609 (170.43 MB)
_________________________________________________________________


In [5]:
with tf.device("/GPU:0"):
    history= model.fit(
        datagen.flow(X_train, y_train, batch_size= 32),
        epochs= 5, 
        validation_data= (X_test, y_test)
    )

Epoch 1/5


2024-08-28 22:37:42.335816: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
W0000 00:00:1724864862.390674   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.450168   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.453921   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.458353   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.462627   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.468851   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.473221   31484 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864862.510921   31484 gpu_t

13/15 [=========================>....] - ETA: 0s - loss: 0.5590 - accuracy: 0.7885

W0000 00:00:1724864867.049861   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.052688   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.055543   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.059543   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.063133   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.066931   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.070761   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.077661   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864867.085444   31487 gp

14/15 [===========================>..] - ETA: 0s - loss: 0.5273 - accuracy: 0.8031

...

15/15 [==============================] - ETA: 0s - loss: 0.5012 - accuracy: 0.8142

W0000 00:00:1724864869.805898   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.808994   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.811574   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.814688   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.817742   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.820995   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.824301   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.830169   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864869.836296   31487 gp

15/15 [==============================] - 12s 448ms/step - loss: 0.5012 - accuracy: 0.8142 - val_loss: 0.0421 - val_accuracy: 1.0000


W0000 00:00:1724864870.420191   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.477182   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.478744   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.480147   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.481600   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.483140   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.484448   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.485739   31487 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724864870.487467   31487 gp

Epoch 2/5
15/15 [==============================] - 4s 279ms/step - loss: 0.0579 - accuracy: 0.9833 - val_loss: 0.0169 - val_accuracy: 1.0000
Epoch 3/5
15/15 [==============================] - 4s 277ms/step - loss: 0.0247 - accuracy: 0.9958 - val_loss: 0.0069 - val_accuracy: 1.0000
Epoch 4/5
15/15 [==============================] - 5s 294ms/step - loss: 0.0186 - accuracy: 0.9979 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 5/5
15/15 [==============================] - 4s 277ms/step - loss: 0.0163 - accuracy: 0.9958 - val_loss: 0.0019 - val_accuracy: 1.0000


In [6]:
model.save("H5/resnet.h5")

/home/manankhatri/.local/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 1s 157ms/step - loss: 0.0019 - accuracy: 1.0000
Test Accuracy: 100.00%
